In [1]:
import pandas as pd
import numpy as np
import os
import openai
import dotenv
from openai import OpenAI
import sys
import pandas as pd
import json
from dotenv import load_dotenv
import re

In [2]:
df = pd.read_csv('..//datasets//final_dataset_v2.csv', low_memory=False)

In [3]:
df

,id,data,entradeta,permatitle,avantitol,tematiques,tags,titol,titol_portada,comarques,cos,redactor,imatges,tweets,quotes
0,3069314,01/01/2021,"La festa ha acabat amb tranquil·litat, però ah...","Enfrontaments amb la policia en una ""rave"" a F...",Lieuron,"[{'id': 'PUACT_MON', 'desc': 'Món', 'main': Tr...","[{'id': 'FRANCA', 'desc': 'França', 'main': Tr...",Almenys 5 detinguts i més de 1.600 multes en e...,NaN,NaN,"<p>Després de 36 hores de festa, la gendarmeri...",Redacció,[{'text': 'https://img.3cat.cat/multimedia/jpg...,Je salue l'action des services de l'Etat et de...,"""Sabíem a què ens arriscàvem, però havíem de f..."
1,3069315,01/01/2021,Entra en vigor la normativa que prohibeix la c...,"Els patinets elèctrics, fora de voreres i zone...",NaN,"[{'id': 'PUACT_SOCIET', 'desc': 'Societat', 'm...","[{'id': 'TRANSIT', 'desc': 'Trànsit', 'main': ...","Els patinets elèctrics, fora de voreres i zone...",NaN,NaN,"<p>Des d'aquest dissabte, els patinets elèctri...",NaN,[{'text': 'https://img.3cat.cat/multimedia/jpg...,NaN,"""Obrirem una segona fase de debat amb els ajun..."
2,3067714,01/01/2021,El 23 de juliol s'ha d'encendre a l'Estadi Nac...,Els Jocs de Tòquio es faran l'estiu del 2021 c...,Jocs Olímpics,"[{'id': 'PUE3_ALTRES', 'desc': 'Més Esports', ...","[{'id': 'JOCSOLIMPICS', 'desc': 'Jocs Olímpics...",Els Jocs de Tòquio es faran l'estiu del 2021 c...,"Tòquio 2020+1, la flama de l'esperança",NaN,"<p>El 24 de juliol, a les 9 del vespre, Tòquio...",Marc Negre,[{'text': 'https://img.3cat.cat/multimedia/jpg...,The Olympic Games #Tokyo2020 will be held from...,"""He proposat posposar un any els Jocs Olímpics..."
3,3069308,01/01/2021,"""La música té la missió de fer una societat mi...",Muti reivindica la importància de la cultura e...,NaN,"[{'id': 'PUACT_CULTUR', 'desc': 'Cultura (Actu...","[{'id': 'MUSICA', 'desc': 'Música', 'main': Fa...",Muti reivindica la importància de la cultura e...,NaN,NaN,<p>El cèlebre director Riccardo Muti <strong>h...,NaN,[{'text': 'https://img.3cat.cat/multimedia/jpg...,NaN,"""Els músics tenen flors a les seves armes, no ..."
4,3069312,01/01/2021,NaN,Els Mossos avisen als participants de la 'rave...,NaN,NaN,NaN,Els Mossos avisen als participants de la 'rave...,NaN,"[{'id': 'PUCOM_VALLOR', 'desc': 'Vallès Orient...",<p>ACN Barcelona.-Els Mossos d'Esquadra avisen...,NaN,[{'text': 'https://img.3cat.cat/multimedia/jpg...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147088,3324430,30/11/2024,"El programa ""Valor afegit"" repassa tres projec...",Empreses contra l'edatisme que lluiten per la ...,NaN,"[{'id': 'PUACT_SOCIET', 'desc': 'Societat', 'm...","[{'id': 'EMPRESA', 'desc': 'Empresa', 'main': ...",Empreses contra l'edatisme que lluiten per la ...,NaN,NaN,<p>Les empreses estan obligades legalment a ev...,Toni Canyís,[{'text': 'https://img.3cat.cat/multimedia/png...,NaN,"""Per primera vegada, em vaig trobar que ja no ..."
147089,3324375,30/11/2024,"Creat el 2016, ja conté dades de 100 milions d...","L'Atles Cel·lular Humà, un projecte per fer pa...",NaN,"[{'id': 'PUACT_SOCIET', 'desc': 'Societat', 'm...","[{'id': 'SALUT', 'desc': 'Salut', 'main': Fals...","L'Atles Cel·lular Humà, un projecte per fer pa...",NaN,NaN,"<p style=""margin-top:0cm;"">Aquest mes l'<stron...",NaN,[{'text': 'https://img.3cat.cat/multimedia/jpg...,The Human Cell Atlas is yielding detailed maps...,"""L'Atles Cel·lular Humà ofereix mapes detallat..."
147090,3324553,30/11/2024,El congrés socialista debatrà 400 esmenes a la...,"Congrés PSOE: El PSC, còmode en el model de fi...",NaN,"[{'id': 'PUACT_POLITI', 'desc': 'Política', 'm...","[{'id': 'PSC', 'desc': 'PSC', 'main': False, '...","Congrés PSOE: el PSC, còmode en el model de fi...",NaN,NaN,<p>Els socialistes catalans arriben al 41è con...,Laura Catalan Asenjo,[{'text': 'https://img.3cat.cat/multimedia/jpg...,NaN,NaN
147091,3323597,30/11/2024,"A les set de la tarda, més del 78% dels milita...",Esquerra decideix avui el seu rumb: les claus ...,NaN,"[{'id': 'PUA

In [4]:
# Carreguem la API KEY
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = openai_api_key

# Models disponibles
models = openai.models.list()
model_list = [model.id for model in models.data]
print(model_list)

['gpt-4o-audio-preview-2024-12-17', 'dall-e-3', 'dall-e-2', 'gpt-4o-audio-preview-2024-10-01', 'text-embedding-3-small', 'o3', 'o3-2025-04-16', 'o4-mini', 'gpt-4.1-nano', 'gpt-4.1-nano-2025-04-14', 'gpt-4o-realtime-preview-2024-10-01', 'o4-mini-2025-04-16', 'gpt-4o-realtime-preview', 'babbage-002', 'gpt-4', 'text-embedding-ada-002', 'text-embedding-3-large', 'gpt-4o-mini-audio-preview', 'gpt-4o-audio-preview', 'o1-preview-2024-09-12', 'gpt-4o-mini-realtime-preview', 'gpt-4.1-mini', 'gpt-4o-mini-realtime-preview-2024-12-17', 'gpt-3.5-turbo-instruct-0914', 'gpt-4o-mini-search-preview', 'gpt-4.1-mini-2025-04-14', 'computer-use-preview-2025-03-11', 'chatgpt-4o-latest', 'davinci-002', 'gpt-3.5-turbo-1106', 'gpt-4o-search-preview', 'gpt-4-turbo', 'gpt-4o-realtime-preview-2024-12-17', 'gpt-3.5-turbo-instruct', 'gpt-3.5-turbo', 'gpt-4-turbo-preview', 'gpt-4o-mini-search-preview-2025-03-11', 'gpt-4-0125-preview', 'gpt-4o-2024-11-20', 'whisper-1', 'gpt-4o-2024-05-13', 'gpt-4-turbo-2024-04-09', '

In [32]:
template = '''
El següent text és una notícia publicada a 3Cat, llegeix-la i respon a les preguntes següents:

{text}

Tasca: hauràs de determinar si la notícia tracta sobre persones grans (gent gran). Considera que una notícia tracta sobre persones grans si parla sobre temes relacionats amb la vellesa, la gent gran, les persones majors, jubilats, pensions, residències per a gent gran, serveis per a persones grans, salut en la vellesa, discriminació per edat cap a persones grans, o qualsevol situació on les persones grans siguin el grup principal del qual es parla. Considera articles que puguin contenir discriminació cap a la gent gran també.

Si la notícia no parla sobre aquests temes o no es pot saber, classifica-la com "altres".

Instrucció: respondràs amb un json amb el següent format:

{
    "tema": "tema de la notícia"
}

on "tema" pot ser "gent gran" o "altres".
'''

In [51]:
client = OpenAI()


def generate_api_response_model_4o(prompt):
    completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )
    response_content = completion.choices[0].message.content
    return response_content

In [52]:
def generate_prompt(article, template):
    article_titol = article['cos']
    article_titol = str(article_titol)
    prompt = template.replace("{text}", article_titol)
    return prompt

In [53]:
row = df[df['id'] == 3273626].iloc[0]
prompt = generate_prompt(row, template)
response = generate_api_response_model_4o(prompt)
print(response)

```json
{
    "tema": "gent gran"
}
```


In [14]:
def repair_json(json_string):
    # Try to extract JSON (objects or arrays)
    json_match = re.search(r"(\{.*\}|\[.*\])", json_string, re.DOTALL)
    if json_match:
        json_content = json_match.group(0)
        try:
            parsed_json = json.loads(json_content)
            print("Valid JSON after extraction:", parsed_json)
            return parsed_json
        except json.JSONDecodeError as e:
            raise ValueError("No JSON object found in response")
    else:
        #throw error
        raise ValueError("No JSON object found in response")


In [5]:
import pandas as pd

# Asegurarte que la columna 'data' es tipo fecha
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')

# Filtrar a partir de junio de 2024
df_filtrat = df[df['data'] >= '2024-10-01']

# Mostrar las primeras filas para comprobar
print(df_filtrat.head())


             id       data                                          entradeta  \
141582  3314043 2024-10-01  Teheran adverteix Israel d'una resposta "més s...   
141583  3313924 2024-10-01  Mazón diu que vol fomentar l'ús del valencià e...   
141584  3314041 2024-10-01  El davanter del Borussia Dortmund ha aconsegui...   
141585  3314024 2024-10-01  Els treballs, que tenen un pressupost de gaire...   
141586  3314019 2024-10-01  Centenars de persones s'han concentrat a la pl...   

                                               permatitle          avantitol  \
141582  Netanyahu promet que l'Iran pagarà per l'atac ...                NaN   
141583  Títol de valencià automàtic: el govern Mazón e...                NaN   
141584  Un hat-trick que passarà a la història particu...  Lliga de Campions   
141585  Es reprenen les obres a l'estació de l'L9 a Sa...          Barcelona   
141586  L'ANC i els CDR criden a buscar "noves formes ...                NaN   

                                

In [6]:
df_filtrat

,id,data,entradeta,permatitle,avantitol,tematiques,tags,titol,titol_portada,comarques,cos,redactor,imatges,tweets,quotes
141582,3314043,2024-10-01,"Teheran adverteix Israel d'una resposta ""més s...",Netanyahu promet que l'Iran pagarà per l'atac ...,NaN,"[{'id': 'PUACT_MON', 'desc': 'Món', 'main': Tr...","[{'id': 'ISRAEL', 'desc': 'Israel', 'main': Tr...","""Pagarà per l'error"": Netanyahu amenaça l'Iran...",NaN,NaN,<p>El <strong>primer ministre israelià</strong...,Redacció/Agències,[{'text': 'https://img.3cat.cat/multimedia/jpg...,NaN,"""El règim de l'Iran no comprèn la nostra deter..."
141583,3313924,2024-10-01,Mazón diu que vol fomentar l'ús del valencià e...,Títol de valencià automàtic: el govern Mazón e...,NaN,"[{'id': 'PUACT_POLITI', 'desc': 'Política', 'm...","[{'id': 'CATALA', 'desc': 'Català ', 'main': T...",Títol de valencià automàtic: el govern Mazón e...,NaN,NaN,<p>La <strong>Generalitat Valenciana expedirà ...,Redacció,[{'text': 'https://img.3cat.cat/multimedia/png...,"Desde mañana mismo, sin burocracias ni tasas, ...","""No és cap premi, sinó un reconeixement que és..."
141584,3314041,2024-10-01,El davanter del Borussia Dortmund ha aconsegui...,Un hat-trick que passarà a la història particu...,Lliga de Campions,"[{'id': 'PUE3_CHAMPIO', 'desc': 'Futbol - Cham...",NaN,Un hat-trick que passarà a la història particu...,NaN,NaN,<p>La primera part de <strong>Karim Adeyemi se...,Redacció Esport3,[{'text': 'https://img.3cat.cat/multimedia/jpg...,"42' Adeyemi zieht von rechts in die Mitte, häl...",NaN
141585,3314024,2024-10-01,"Els treballs, que tenen un pressupost de gaire...",Es reprenen les obres a l'estació de l'L9 a Sa...,Barcelona,"[{'id': 'PUACT_SOCIET', 'desc': 'Societat', 'm...","[{'id': 'METRO', 'desc': 'Metro', 'main': True...",Es reprenen les obres a l'estació de l'L9 a Sa...,NaN,"[{'id': 'PUCOM_BARCEL', 'desc': 'Barcelonès', ...",<p>Aquest dimarts s'han <strong>reprès les obr...,Redacció,[{'text': 'https://img.3cat.cat/multimedia/jpg...,NaN,NaN
141586,3314019,2024-10-01,Centenars de persones s'han concentrat a la pl...,"L'ANC i els CDR criden a buscar ""noves formes ...",NaN,"[{'id': 'PUACT_POLITI', 'desc': 'Política', 'm...","[{'id': 'REFERENDUM', 'desc': 'Referèndum 1-O'...","L'ANC i els CDR criden a buscar ""noves formes ...",NaN,NaN,"<p>Reivindicar ""la validesa del mandat popular...",Redacció,[{'text': 'https://img.3cat.cat/multimedia/jpe...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147088,3324430,2024-11-30,"El programa ""Valor afegit"" repassa tres projec...",Empreses contra l'edatisme que lluiten per la ...,NaN,"[{'id': 'PUACT_SOCIET', 'desc': 'Societat', 'm...","[{'id': 'EMPRESA', 'desc': 'Empresa', 'main': ...",Empreses contra l'edatisme que lluiten per la ...,NaN,NaN,<p>Les empreses estan obligades legalment a ev...,Toni Canyís,[{'text': 'https://img.3cat.cat/multimedia/png...,NaN,"""Per primera vegada, em vaig trobar que ja no ..."
147089,3324375,2024-11-30,"Creat el 2016, ja conté dades de 100 milions d...","L'Atles Cel·lular Humà, un projecte per fer pa...",NaN,"[{'id': 'PUACT_SOCIET', 'desc': 'Societat', 'm...","[{'id': 'SALUT', 'desc': 'Salut', 'main': Fals...","L'Atles Cel·lular Humà, un projecte per fer pa...",NaN,NaN,"<p style=""margin-top:0cm;"">Aquest mes l'<stron...",NaN,[{'text': 'https://img.3cat.cat/multimedia/jpg...,The Human Cell Atlas is yielding detailed maps...,"""L'Atles Cel·lular Humà ofereix mapes detallat..."
147090,3324553,2024-11-30,El congrés socialista debatrà 400 esmenes a la...,"Congrés PSOE: El PSC, còmode en el model de fi...",NaN,"[{'id': 'PUACT_POLITI', 'desc': 'Política', 'm...","[{'id': 'PSC', 'desc': 'PSC', 'main': False, '...","Congrés PSOE: el PSC, còmode en el model de fi...",NaN,NaN,<p>Els socialistes catalans arriben al 41è con...,Laura Catalan Asenjo,[{'text': 'https://img.3cat.cat/multimedia/jpg...,NaN,NaN
147091,3323597,2024-11-30,"A les set de la tarda, més del 78% dels milita...",Esquerra decideix avui el seu rumb: les clau

In [60]:
results = {}
i = 0
length_df = len(df_filtrat)

for _, row in df_filtrat.iterrows():
    if i % 250 == 0:
        print(f"Processing row {i} of {length_df}")
    i +=1
    prompt = generate_prompt(row, template)
    response = generate_api_response_model_4o(prompt)
    id = row['id']

    # Check if the response is valid JSON
    try:
        response_json = json.loads(response)  # Try to parse the response as JSON
        results[row['id']] = response_json   # Store the parsed JSON under the id
    except json.JSONDecodeError:
        try:
            response_json = repair_json(response)
            results[row['id']] = response_json
        except ValueError as e:
            print("FATAL ERROR: Could not extract JSON from response")
            results[row['id']] = {"error": "Invalid JSON response"}
            break


# Convert the results dictionary to a JSON string
final_json = json.dumps(results, indent=4)
print(final_json)

Processing row 0 of 5511
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON after extraction: {'tema': 'altres'}
Valid JSON afte

In [61]:
# Save to a file
with open('classificacio.json', 'w') as json_file:
    json_file.write(final_json)

print("JSON file has been saved as 'results.json'.")

JSON file has been saved as 'results.json'.


In [7]:
import json

# Load the JSON file
with open('classificacio.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Extract IDs where "tema" is "gent gran"
gent_gran_ids = [id_ for id_, info in data.items() if info.get("tema") == "gent gran"]

print(gent_gran_ids)


['3313907', '3313975', '3313967', '3313881', '3313850', '3313841', '3313098', '3314176', '3314170', '3314113', '3314079', '3314016', '3314419', '3314396', '3314276', '3314304', '3314597', '3314442', '3314435', '3314684', '3314669', '3314640', '3309081', '3314591', '3314776', '3314694', '3314717', '3314908', '3314813', '3314866', '3314857', '3315155', '3315087', '2722262', '3314860', '3314942', '3315474', '3315449', '3315264', '3315292', '3315639', '3315616', '3315681', '3315544', '3315664', '3315662', '3315651', '3315621', '3315545', '3315610', '3314325', '3315893', '3315911', '3315953', '3316088', '3316091', '3315771', '3316228', '3316233', '3316203', '3316201', '3316191', '3316160', '3316146', '3316504', '3316697', '3316617', '3316956', '3316793', '3316827', '3316794', '3317010', '3317064', '3317059', '3317058', '3316978', '3317255', '3317228', '3317188', '3317305', '3317235', '3317512', '3317484', '3317442', '3317399', '3317974', '3317981', '3317924', '3317930', '3318202', '3317910'

In [8]:
import pandas as pd

# Filter the DataFrame
df_gent_gran = df_filtrat[df_filtrat['id'].astype(str).isin(gent_gran_ids)]

print(df_gent_gran)


             id       data                                          entradeta  \
141604  3313907 2024-10-01  Actualment, a les facultats catalanes hi ha 28...   
141620  3313975 2024-10-01                                                NaN   
141628  3313967 2024-10-01                                                NaN   
141656  3313881 2024-10-01                                                NaN   
141675  3313850 2024-10-01  La produccions participades per 3Cat, "Tardes ...   
...         ...        ...                                                ...   
144561  3319796 2024-11-03                                                NaN   
144602  3319964 2024-11-04  L'exèrcit israelià mata una dona palestina a C...   
144607  3319992 2024-11-04  El veterà entrenador dels San Antonio Spurs va...   
144619  3319961 2024-11-04                                                NaN   
144671  3319830 2024-11-04  Va produir el cèlebre "Thriller", va treballar...   

                           

In [11]:
template = '''
El següent és un article periodístic sobre persones grans. Llegeix-lo amb atenció i analitza si s’adequa als criteris del llibre d’estil de la CCMA sobre el tractament de les persones grans (*GE: 1.1.2.5. Principis editorials: atenció a la societat*).

####################
ARTICLE:

{text}

####################

### Tasca:
Avalua si l'article compleix els criteris editorials següents. Justifica cada punt i ofereix una valoració final argumentada.

### Criteris a revisar:

1. **Llenguatge correcte i respectuós**:
   - Es fa servir el terme "persones grans" o "gent gran" de manera adequada?
   - S’evita usar "avis" com a sinònim general de persones grans, excepte quan es parla específicament d’avis i àvies?
   - S’evita el llenguatge paternalista, condescendent o estereotipat?

2. **Representació diversa i activa**:
   - Les persones grans es mostren com a membres actius de la societat (participació social, cultural, laboral…)?
   - No es limiten a aparèixer en contextos de dependència, malaltia o fragilitat?

3. **Absència d’estereotips i dramatització**:
   - Es respecta la dignitat de les persones grans, evitant tòpics com la inutilitat, la solitud extrema o la fragilitat excessiva?
   - S’evita el to compassiu, dramàtic o espectacular injustificat?

4. **Precisió terminològica**:
   - Si es fa servir "avis", és correcte segons el context (persones que tenen néts)?
   - No es generalitza aquest terme per a totes les persones grans?

5. **Bones pràctiques periodístiques**:
   - El llenguatge és inclusiu i no esbiaixat?
   - Es recullen veus pròpies de les persones grans o fonts expertes per aportar context o testimonis?

---

### Resultat esperat:
Respon exclusivament amb un objecte JSON en aquest format:

```json
{
  "reason": "<Explicació raonada en català sobre el grau de compliment dels criteris, amb exemples clars del text analitzat.>",
  "rating": <Un número enter de l'1 al 100, on 1 és gens alineat amb el llibre d’estil i 100 és totalment alineat.>
}



'''

In [ ]:
template_gent_gran = '''
El següent és un article de notícies sobre persones grans. Llegeix-lo amb atenció i realitza la tasca que segueix, tenint com a referència la *Guia d’estil de 3Cat per al tractament de la gent gran als continguts audiovisuals*.

####################
ARTICLE:

{text}

####################

Tasca: Determina si l'article s'adhereix als principis de la guia d’estil sobre el tractament de la gent gran.

Instruccions:

1. **Anàlisi del llenguatge emprat**:
   - Es fa servir correctament el terme "persones grans" o "gent gran"? S'evita l’ús de termes com "vells", "ancians", "avis" (quan no és pertinent), "jubilats" o "tercera edat" de manera generalitzada?
   - El llenguatge evita el paternalisme i la infantilització? Es tracta la gent gran com a persones amb capacitat de decisió?
   - Es reflecteix la diversitat de les persones grans sense estereotips negatius (dependència, fragilitat, malaltia, soledat, etc.)?

2. **Representació visual i narrativa**:
   - Si es descriuen imatges o vídeos, es defugen els estereotips visuals sobre la gent gran (imatges de soledat, malaltia o inactivitat)?
   - Es mostren relats positius sobre la vida activa, la participació social, el coneixement, l’experiència i les aportacions de les persones grans?
   - No es limiten a aparèixer en contextos de dependència, malaltia o fragilitat?
   - Es respecta la dignitat de les persones grans, evitant tòpics com la inutilitat, la solitud extrema o la fragilitat excessiva?

3. **Contextualització social i estructural**:
   - Es presenta l’envelliment com un procés vital natural?
   - Es destaca el paper actiu de les persones grans a la societat i la seva contribució en diferents àmbits?
   - S’evita associar les persones grans exclusivament amb la dependència o la malaltia?

4. **Perspectiva de gènere i diversitat**:
   - Es té en compte la perspectiva de gènere? Es visibilitzen les dones grans i s’evita la doble invisibilització per raó de gènere i edat?
   - Es reflecteix la diversitat (cultural, social, funcional) dins el col·lectiu de la gent gran?

5. **Participació i veu pròpia**:
   - S’inclou la veu de les pròpies persones grans com a protagonistes? Apareixen testimonis o opinions d’aquest col·lectiu?
   - S’evita parlar *sobre* les persones grans sense donar-los la paraula?

6. **Altres bones pràctiques periodístiques**:
   - Es potencia la participació de la gent gran en espais d’opinió i programes de debat o informació?
   - Es promouen iniciatives, projectes o activitats en què participen persones grans, fugint de la mirada assistencialista?

**Important**: Tingues en compte que **no tots els criteris són aplicables en cada cas**. L’anàlisi ha de tenir en compte el tipus de notícia, el moment informatiu (primeres hores vs. seguiment), i la naturalesa del cas. Valora només els criteris rellevants segons el context.

**Conclusió**: A partir dels punts analitzats i segons el context informatiu, determina si l’article s'adapta als punts mencionats. Justifica amb claredat els punts forts i les mancances observades, i proporciona una valoració final fonamentada.

**Sortida esperada**: Retorna únicament un objecte JSON amb aquest format:

1. "reason": "<Resum del raonament en català. Ha de reflectir el teu pensament pas a pas i la justificació final.>",
2. "rating": <Un número enter de l'1 al 100 indicant el grau d'alineació amb la guia d’estil, on 1 és gens alineat i 100 és totalment alineat.>

'''


In [10]:
client = OpenAI()


def generate_api_response_model_4o(prompt):
    completion = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )
    response_content = completion.choices[0].message.content
    return response_content

In [11]:
def generate_prompt(article, template):
    article_titol = article['cos']
    article_titol = str(article_titol)
    prompt = template.replace("{text}", article_titol)
    return prompt

In [15]:
results = {}
i = 0
length_df = len(df_gent_gran)

for _, row in df_gent_gran.iterrows():
    if i % 250 == 0:
        print(f"Processing row {i} of {length_df}")
    i +=1
    prompt = generate_prompt(row, template_gent_gran)
    response = generate_api_response_model_4o(prompt)
    id = row['id']

    # Check if the response is valid JSON
    try:
        response_json = json.loads(response)  # Try to parse the response as JSON
        results[row['id']] = response_json   # Store the parsed JSON under the id
    except json.JSONDecodeError:
        try:
            response_json = repair_json(response)
            results[row['id']] = response_json
        except ValueError as e:
            print("FATAL ERROR: Could not extract JSON from response")
            results[row['id']] = {"error": "Invalid JSON response"}
            break


# Convert the results dictionary to a JSON string
final_json = json.dumps(results, indent=4)
print(final_json)

Processing row 0 of 152
Valid JSON after extraction: {'reason': "L'article fa un ús adequat del terme 'gent gran' i evita l'ús de termes com 'vells' o 'ancians'. No hi ha un to paternalista ni infantilitzador i es tracten les persones grans com a individus capaços de prendre decisions, com es demostra en la narrativa de la Maria Antònia que continua estudiant a la universitat per decisió pròpia. L'article evita estereotips negatius i mostra una imatge positiva de la Maria Antònia com una persona activa i amb ganes d'aprendre. La presentació de la seva trajectòria és un relat positiu que destaca la seva participació activa en l'educació superior. El text reconeix explícitament la problemàtica de l'edatisme en la sanitat, el que demostra certa consideració pel context social i estructural. Es visibilitza una dona gran, la Maria Antònia, com a protagonista de la seva història, oferint la seva veu i perspectiva al llarg de l'article. No obstant això, l'article podria haver esmentat més la 

In [16]:
# Convert the results dictionary to a JSON string
final_json = json.dumps(results, indent=4)
print(final_json)

{
    "3313907": {
        "reason": "L'article fa un \u00fas adequat del terme 'gent gran' i evita l'\u00fas de termes com 'vells' o 'ancians'. No hi ha un to paternalista ni infantilitzador i es tracten les persones grans com a individus capa\u00e7os de prendre decisions, com es demostra en la narrativa de la Maria Ant\u00f2nia que continua estudiant a la universitat per decisi\u00f3 pr\u00f2pia. L'article evita estereotips negatius i mostra una imatge positiva de la Maria Ant\u00f2nia com una persona activa i amb ganes d'aprendre. La presentaci\u00f3 de la seva traject\u00f2ria \u00e9s un relat positiu que destaca la seva participaci\u00f3 activa en l'educaci\u00f3 superior. El text reconeix expl\u00edcitament la problem\u00e0tica de l'edatisme en la sanitat, el que demostra certa consideraci\u00f3 pel context social i estructural. Es visibilitza una dona gran, la Maria Ant\u00f2nia, com a protagonista de la seva hist\u00f2ria, oferint la seva veu i perspectiva al llarg de l'article

In [17]:
# Save to a file
with open('results2.json', 'w') as json_file:
    json_file.write(final_json)

print("JSON file has been saved as 'results.json'.")

JSON file has been saved as 'results.json'.


In [18]:
import pandas as pd
import json

# Path to your JSON file
file_path = 'results.json'
file_path_2 = 'results2.json'

# Load the JSON data
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

with open(file_path_2, 'r', encoding='utf-8') as f:
    data_2 = json.load(f)



# Convert the JSON dictionary into a DataFrame
df = pd.DataFrame.from_dict(data, orient='index')
df_2 = pd.DataFrame.from_dict(data_2, orient='index')

# If you want to reset the index and have the keys (IDs) as a column:
df = df.reset_index().rename(columns={'index': 'article_id'})
df_2 = df_2.reset_index().rename(columns={'index': 'article_id'})

# Show the DataFrame
print(df)


   article_id                                             reason  rating
0     3313907  El llenguatge utilitzat en l'article és genera...      90
1     3313975  L'article compleix en gran part els criteris e...      95
2     3313967  L'article compleix majoritàriament els criteri...      85
3     3313881  L'article no compleix plenament amb les direct...      40
4     3313850  L'article proporciona una visió general de les...      50
..        ...                                                ...     ...
90    3317910  L'article compleix els criteris del llibre d'e...      95
91    3317950  L'article analitzat està centrat en la menopau...      80
92    3318176  1. Llenguatge correcte i respectuós: L'article...      85
93    3318171  L'article presenta diverses mancances en relac...      60
94    3318066  1. Llenguatge correcte i respectuós: L'article...      80

[95 rows x 3 columns]


In [19]:
df

,article_id,reason,rating
0,3313907,El llenguatge utilitzat en l'article és genera...,90
1,3313975,L'article compleix en gran part els criteris e...,95
2,3313967,L'article compleix majoritàriament els criteri...,85
3,3313881,L'article no compleix plenament amb les direct...,40
4,3313850,L'article proporciona una visió general de les...,50
...,...,...,...
90,3317910,L'article compleix els criteris del llibre d'e...,95
91,3317950,L'article analitzat està centrat en la menopau...,80
92,3318176,1. Llenguatge correcte i respectuós: L'article...,85
93,3318171,L'article presenta diverses mancances en relac...,60


In [20]:
df_2

,article_id,reason,rating
0,3313907,L'article fa un ús adequat del terme 'gent gra...,85
1,3313975,L'article utilitza correctament el terme 'pers...,85
2,3313967,L'article utilitza correctament el terme 'pers...,75
3,3313881,L'article no compleix de manera adequada amb l...,40
4,3313850,L'article analitzat tracta principalment sobre...,60
...,...,...,...
147,3319796,L'article es refereix correctament a les 'pers...,65
148,3319964,L'article tracta sobre un incident tràgic a Ci...,75
149,3319992,1. Anàlisi del llenguatge emprat: L'article s'...,85
150,3319961,L'article s'adhereix en part als principis de ...,70


In [20]:
# Sort the DataFrame by 'rating' in ascending order
lowest_rated = df.sort_values(by='rating', ascending=True).head(3)

# Print the article_id, rating, and reason
for idx, row in lowest_rated.iterrows():
    print(f"Article ID: {row['article_id']}")
    print(f"Rating: {row['rating']}")
    print(f"Reason: {row['reason']}\n")
    print('-' * 80)



Article ID: 3316201
Rating: 20
Reason: L'article analitzat no compleix gairebé cap dels criteris editorials establerts per al tractament de persones grans segons el llibre d'estil de la CCMA. 

1. **Llenguatge correcte i respectuós**: L'article no esmenta explícitament les persones grans, ni fa referència a elles utilitzant termes com 'persones grans', 'gent gran', o 'avis'. La peça està centrada en qüestions sobre la construcció i compravenda d'habitatges, sense mencionar aquest grup poblacional. Per tant, no hi ha llenguatge paternalista, condescendent o estereotipat envers les persones grans, ja que no forma part del focus de l'article.

2. **Representació diversa i activa**: De nou, l'article no aborda cap cas de persones grans, ja sigui per mostrar-les com a membres actius de la societat o en contextos de dependència. No es menciona cap exemple de participació social, cultural o laboral de persones grans.

3. **Absència d’estereotips i dramatització**: Com que no es fa referència 

In [19]:
# Sort the DataFrame by 'rating' in ascending order
lowest_rated = df_2.sort_values(by='rating', ascending=True).head(3)

# Print the article_id, rating, and reason
for idx, row in lowest_rated.iterrows():
    print(f"Article ID: {row['article_id']}")
    print(f"Rating: {row['rating']}")
    print(f"Reason: {row['reason']}\n")
    print('-' * 80)



Article ID: 3318989
Rating: 10
Reason: L'article no s'adhereix als principis de la guia d'estil sobre el tractament de la gent gran pels motius següents: 

1. **Anàlisi del llenguatge emprat**: L'article utilitza un llenguatge que pot ser percebut com paternalista i infantilitzant, al referir-se als veïns que expressen preocupacions sobre l'activitat nocturna amb termes com 'rondinaires', 'xerifs' i 'petits dictadors'. A més, la frase 'potser ells han envellit malament i han digerit malament una cosa com és fer-se gran' implica un judici negatiu sobre l'envelliment, que no reflecteix la diversitat ni evita estereotips. 

2. **Representació visual i narrativa**: No es descriuen imatges ni vídeos a l'article, però la narració expressada pel director del Gremi de Restauració de Barcelona conté una visió negativa de les persones grans, sense relats positius sobre la seva contribució social o vital. 

3. **Contextualització social i estructural**: L'envelliment no es presenta com un procés 

In [38]:
# Get the 15 least-rated article IDs from df
least_15_df = df.sort_values(by='rating', ascending=True).head(50)['article_id']

# Get the 15 least-rated article IDs from df_2
least_15_df2 = df_2.sort_values(by='rating', ascending=True).head(50)['article_id']

# Convert to sets for easier comparison
set_df = set(least_15_df)
set_df2 = set(least_15_df2)

# Find common IDs
common_ids = set_df.intersection(set_df2)

# Print results
print("IDs from df (15 least rated):", sorted(set_df))
print("IDs from df_2 (15 least rated):", sorted(set_df2))
print("Repeated IDs in both:", sorted(common_ids))


IDs from df (15 least rated): ['2722262', '3309081', '3313850', '3313881', '3314113', '3314170', '3314176', '3314276', '3314304', '3314396', '3314419', '3314442', '3314597', '3314640', '3314776', '3314857', '3314908', '3314942', '3315087', '3315155', '3315264', '3315292', '3315449', '3315545', '3315610', '3315621', '3315651', '3315662', '3315893', '3315953', '3316091', '3316146', '3316191', '3316201', '3316203', '3316228', '3316793', '3316794', '3316827', '3317059', '3317228', '3317255', '3317399', '3317442', '3317950', '3317974', '3317981', '3318066', '3318171', '3318202']
IDs from df_2 (15 least rated): ['2722262', '3313881', '3314113', '3314170', '3314396', '3314442', '3314591', '3314640', '3314813', '3314857', '3314908', '3314942', '3315087', '3315155', '3315264', '3315292', '3315449', '3315651', '3315662', '3315664', '3315893', '3315953', '3316146', '3316191', '3316233', '3316794', '3316978', '3317010', '3317059', '3317228', '3317255', '3317442', '3317861', '3317981', '3318176', '

In [52]:
# Get the reason from df
reason_df = df.loc[df['article_id'] == "3316228", 'reason'].values[0]

# Get the reason from df_2
reason_df2 = df_2.loc[df_2['article_id'] == "3317442", 'reason'].values[0]

# Print both reasons
print("Reason from df:")
print(reason_df)
print("\nReason from df_2:")
print(reason_df2)


Reason from df:
L'article presenta alguns aspectes que no compleixen completament els criteris del llibre d'estil de la CCMA respecte al tractament de les persones grans. En primer lloc, no s'utilitza el terme 'persones grans' ni 'gent gran'; l'article individualitza el cas a la Maria Rosa Lladó Mora, sense fer ús del terme 'avis', tot i que es podria millorar fent referència de manera més inclusiva a les persones grans. Pel que fa al llenguatge, a moments l'article podria suggerir un to lleugerament condescendent quan es descriu la vulnerabilitat de la Maria Rosa ('tenia por del futur'), i això pot implicar una falta de respecte subtil cap a la seva capacitat de decisió. En termes de representació, la imatge que es presenta és d'una persona vulnerable i víctima d'enganys, afavorint la percepció de fragilitat. Això es reforça amb frases com 'Ella tenia por del futur'. Tanmateix, no hi ha to dramàtic ni compassiu injustificat, i l'article es basa en fets concrets i una problemàtica soci

In [51]:
# Filter the row where 'id' is 3315651 and print the 'cos' column
cos_value = df_gent_gran.loc[df_gent_gran['id'] == "3314176", 'permatitle'].values[0]
print(cos_value)


Un consorci europeu liderat per Vall d'Hebron utilitzarà la IA per millorar tota l'atenció dels pacients amb ictus


In [48]:
import pandas as pd

# Asegúrate de que ambas columnas 'article_id' y 'id' son strings
df['article_id'] = df['article_id'].astype(str)
df_gent_gran['id'] = df_gent_gran['id'].astype(str)

# Ordenar por 'rating' y obtener los primeros 50 'article_id'
least_50_ids = df.sort_values(by='rating', ascending=True).head(50)['article_id']

# Recorremos cada ID y mostramos el 'permatitle'
for article_id in least_50_ids:
    cos_value = df_gent_gran.loc[df_gent_gran['id'] == article_id, 'permatitle'].values
    if len(cos_value) > 0:
        print(f'ID "{article_id}": {cos_value[0]}')
    else:
        print(f'ID "{article_id}": No encontrado en df_gent_gran')


ID "3316201": Osona encapçala l'obra nova a la Catalunya Central amb la finalització de 194 habitatges el primer semestre de 2024
ID "3315651": Els Mossos conclouen que els dos ancians assassinats a Barcelona el 2019 no es van poder defensar
ID "3314908": Un tribunal popular comença a jutjar un comercial per enganyar i assassinar un matrimoni de la tercera edat a Barcelona
ID "3315662": Els Mossos conclouen que els dos ancians assassinats a Barcelona el 2019 no es van poder defensar
ID "3314597": Coralie Fargeat porta a Sitges 'La sustancia', una reflexió sobre la dictadura de la bellesa i la fama
ID "3316146": L'acusat de matar dos avis a Barcelona l'any 2019 nega els fets "rotundament" i diu que eren "bellíssimes persones"
ID "3316191": L'acusat de matar dos avis a Barcelona l'any 2019 nega els fets "rotundament" i diu que eren "bellíssimes persones"
ID "3313881": TerrorMolins acull l'estrena a l'Estat de 'La Damnée' d'Abel Danan en la seva jornada inaugural
ID "3315449": L'acusat de

C:\Users\marti\AppData\Local\Temp\ipykernel_12300\1451174691.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gent_gran['id'] = df_gent_gran['id'].astype(str)
